# Functional Programming and Why It's Relevant for HEP: Exercises (2)

In [1]:
:! stack install lazy-csv

Cabal file warning in /opt/IHaskell/ihaskell-display/ihaskell-aeson/ihaskell-aeson.cabal@46:28: Packages with 'cabal-version: 1.12' or later should specify a specific version of the Cabal spec of the form 'cabal-version: x.y'. Use 'cabal-version: 1.16'.
Cabal file warning in /opt/IHaskell/ihaskell-display/ihaskell-blaze/ihaskell-blaze.cabal@49:28: Packages with 'cabal-version: 1.12' or later should specify a specific version of the Cabal spec of the form 'cabal-version: x.y'. Use 'cabal-version: 1.16'.
Cabal file warning in /opt/IHaskell/ihaskell-display/ihaskell-charts/ihaskell-charts.cabal@46:28: Packages with 'cabal-version: 1.12' or later should specify a specific version of the Cabal spec of the form 'cabal-version: x.y'. Use 'cabal-version: 1.16'.
Cabal file warning in /opt/IHaskell/ihaskell-display/ihaskell-diagrams/ihaskell-diagrams.cabal@46:28: Packages with 'cabal-version: 1.12' or later should specify a specific version of the Cabal spec of the form 'cabal-version: x.y'. Use

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not 
inside code cells, or use for mathematical formulas if needed.
</div>

In [ ]:
import Text.CSV.Lazy.String -- Required for lazily reading CSV files

To make things a bit more readable, lets first define some type aliases and data types to describe a particle:

In [3]:
type Pt = Float
type Eta = Float
type Phi = Float
type Mass = Float
type Charge = Int
data Particle = Particle Pt Eta Phi Mass Charge
                deriving Show

TODO: **very** brief mention of `IO` monad, emphasize it's *just there* but students shouldn't concern themselves with it (illustrated by the following methods that don't need it).

In [4]:
events :: IO [(CSVRow, CSVRow)]
events = do
    content <- readFile "data/df014_CsvDataSource_MuRun2010B.csv"
    let e = csvTable $ parseCSV content
    let m1 = case selectFields ["pt1", "eta1", "phi1", "M", "Q1"] e of
                Left _    -> error "some fields not found"
                Right res -> drop 1 res
    let m2 = case selectFields ["pt2", "eta2", "phi2", "M", "Q2"] e of
                Left _    -> error "some fields not found"
                Right res -> drop 1 res
    return $ zip m1 m2

muon :: CSVRow -> Particle
muon r = let [fPt, fEta, fPhi, fMass, fCharge] = map csvFieldContent r
          in Particle (rf fPt) (rf fEta) (rf fPhi) (rf fMass) (ri fCharge)
          where rf = read :: String -> Float
                ri = read :: String -> Int

dimuon :: (CSVRow, CSVRow) -> (Particle, Particle)
dimuon (r1, r2) = (muon r1, muon r2)

dimuons :: [(CSVRow, CSVRow)] -> [(Particle, Particle)]
dimuons = map dimuon

: 

In [5]:
head . dimuons <$> events

: 

Just in case you don't have a background in physics, for calculating the invariant mass of a set of two particles, we can use the following formula:
$$
M = \sqrt{2p_{T1}p_{T2}(\cosh(\eta_1-\eta_2)-\cos(\phi_1-\phi_2))}
$$

In [ ]:
invMass :: Particle -> Particle -> Float
invMass p1 p2 = let Particle pt1 eta1 phi1 _ _ = p1
                    Particle pt2 eta2 phi2 _ _ = p2
                in sqrt $ (2 * pt1 * pt2) * (cosh (eta1 - eta2) - cos (phi1 - phi2))

In [ ]:
head . dimuons <$> events

: 

In [1]:
take 10 . map (uncurry invMass) . dimuons <$> events

: 

In [ ]:
-- :option no-pager
-- import IHaskell.Display
-- import Graphics.Rendering.Chart.Backend.Diagrams

-- import Graphics.Rendering.Chart.Easy

-- toRenderable (do
--     layout_title .= "f"
--     layout_x_axis . laxis_title .= "Redshift"
--     layout_y_axis . laxis_title .= "f(z)"
--     plot (line "O_matter=0.3 O_lambda=0.7" [zip zs gs])
--     )